<a href="https://colab.research.google.com/github/Vladzimir-40/Proj_1/blob/main/my_file3/dz_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install kagglehub

from google.colab import files
files.upload() # Загрузите файл kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vladimir401011","key":"e6b9eec8a00a0a7c2c3e5fcbd033ab06"}'}

In [3]:
import pandas as pd
import kagglehub
import os

class Data_loading:
    @staticmethod
    def load_from_kaggle(dataset_path, filename):
        """
        Загружает dataset из Kaggle используя kagglehub

        Args:
            dataset_path (str): путь к dataset на Kaggle
            filename (str): название файла для загрузки

        Returns:
            pd.DataFrame: загруженный датафрейм
        """
        download_path = kagglehub.dataset_download(dataset_path)
        file_path = os.path.join(download_path, filename)
        return pd.read_csv(file_path)


import kagglehub

# Download latest version
path = kagglehub.dataset_download("zadafiyabhrami/global-crocodile-species-dataset")

print("Path to dataset files:", path)
print("\nЗагруженные файлы:")
for file in os.listdir(path):
    print(f"📁 {file}")


100%|██████████| 56.1k/56.1k [00:00<00:00, 32.0MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/zadafiyabhrami/global-crocodile-species-dataset/versions/1

Загруженные файлы:
📁 crocodile_dataset.csv


Плучаем датафрейм через статический метод, в который передем "путь" и "имя csv-файла"

In [4]:
df = Data_loading.load_from_kaggle("zadafiyabhrami/global-crocodile-species-dataset", "crocodile_dataset.csv")

Using Colab cache for faster access to the 'global-crocodile-species-dataset' dataset.


In [6]:
# Просматриваем первые несколько строк датасета
print(df.head())

# Получаем общую информацию о датасете
print(df.info())

   Observation ID                         Common Name         Scientific Name  \
0               1                 Morelet's Crocodile    Crocodylus moreletii   
1               2                  American Crocodile       Crocodylus acutus   
2               3                   Orinoco Crocodile  Crocodylus intermedius   
3               4                 Morelet's Crocodile    Crocodylus moreletii   
4               5  Mugger Crocodile (Marsh Crocodile)    Crocodylus palustris   

         Family       Genus  Observed Length (m)  Observed Weight (kg)  \
0  Crocodylidae  Crocodylus                 1.90                  62.0   
1  Crocodylidae  Crocodylus                 4.09                 334.5   
2  Crocodylidae  Crocodylus                 1.08                 118.2   
3  Crocodylidae  Crocodylus                 2.42                  90.4   
4  Crocodylidae  Crocodylus                 3.75                 269.4   

  Age Class      Sex Date of Observation Country/Region      Habitat

Создание базы данных

In [ ]:
import sqlite3

# Подключение к SQLite базе данных (или создание, если не существует)
conn = sqlite3.connect('crocodiles.db')

Делаем таблицу

In [9]:
# Создание курсора
cursor = conn.cursor()

# Создание таблицы "crocodile_inhabitants" (содержит всю информацию об местах обитания крокодилов)
cursor.execute('''CREATE TABLE IF NOT EXISTS crocodile_inhabitants (
    crocodile_id INTEGER PRIMARY KEY,
    common_name VARCHAR(30) NOT NULL,
    scient_name VARCHAR(30) NOT NULL,
    observ_length_m NUMERIC,
    observ_weight_kg NUMERIC,
    age_class VARCHAR(20),
    sex VARCHAR(10),
    date_of_obs VARCHAR(10),
    country_region VARCHAR(30),
    habitat_type VARCHAR(30)
)''')

conn.commit()

Делаем вставку в БД crocodiles.db, в таблицу данных crocodile_inhabitants из df

In [10]:
cursor = conn.cursor()

data_f = df[['Observation ID', 'Common Name', 'Scientific Name', 'Observed Length (m)',
             'Observed Weight (kg)', 'Age Class', 'Sex', 'Date of Observation',
             'Country/Region', 'Habitat Type']].values.tolist()

cursor.executemany('''INSERT INTO crocodile_inhabitants
                   (crocodile_id, common_name, scient_name, observ_length_m,
                    observ_weight_kg, age_class, sex, date_of_obs,
                    country_region, habitat_type)
                   VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', data_f)

conn.commit()